## Objective
Run some naive models (both in terms of feature selection and hyperparameter tuning). This will set benchmark scores that more robustly designed models should outperform. 

In [1]:
%run data_package_loading.py # Code loads data as well as packages that are relevant across most project phases
%matplotlib inline


# !conda install -y psycopg2

from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel, RFECV 
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

Xdb_1 = pd.read_pickle('data/madelon_db_1')
Xdb_2 = pd.read_pickle('data/madelon_db_2')
Xdb_3 = pd.read_pickle('data/madelon_db_3')


ydb_1 = Xdb_1['target']
ydb_2 = Xdb_2['target']
ydb_3 = Xdb_3['target']
Xdb_1 = Xdb_1.drop(['_id', 'target'], axis=1)
Xdb_2 = Xdb_2.drop(['_id', 'target'], axis=1)
Xdb_3 = Xdb_3.drop(['_id', 'target'], axis=1)


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

In [3]:
data_target = [(Xuci_1, yuci_1, 'uci_1'), 
               (Xuci_2, yuci_2, 'uci_2'), 
               (Xuci_3, yuci_3, 'uci_3'), 
               (Xdb_1,  ydb_1, 'db_1'), 
               (Xdb_2,  ydb_2, 'db_2'), 
               (Xdb_3,  ydb_3, 'db_3')]

In [4]:
C = 10 ** 9
model_list = [LogisticRegression(C = C), 
              DecisionTreeClassifier(), 
              KNeighborsClassifier(n_jobs=-1), 
              SVC(C = C)]

In [5]:
def run_pipe_cv (X, y, data_name, model):
    
#     # Test Train split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    # Set up Pipeline
    pipe = Pipeline([
                  ('scaler', StandardScaler()),
                  ('model', model)
                  ])
    
    # Set up Kfolds split
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
    skf.get_n_splits(X, y)
    
    # Run pipe on Kfold
    train_scores = []
    test_scores = []
    for train_cv_index, val_cv_index in skf.split(X, y):
        X_train_temp = X.iloc[train_cv_index, :]
        y_train_temp = y[train_cv_index]
        X_test_temp = X.iloc[val_cv_index, :]
        y_test_temp = y[val_cv_index]
        
        pipe.fit(X_train_temp, y_train_temp)
        train_scores.append(pipe.score(X_train_temp, y_train_temp))
        test_scores.append(pipe.score(X_test_temp, y_test_temp))

    results.append({
                'test_train': 'train',
                'data': data_name,
                'scaler': pipe.named_steps['scaler'],
                'model': pipe.named_steps['model'],
                'score': np.mean(train_scores) #accuracy for classifications?
               })
    results.append({
            'test_train': 'test',
            'data': data_name,
            'scaler': pipe.named_steps['scaler'],
            'model': pipe.named_steps['model'],
            'score': np.mean(test_scores) #accuracy for classifications?
           })
        
    # How do I run the model on the test data?

In [ ]:
results = []

for Xy in data_target:
    for model in model_list:
        run_pipe_cv(Xy[0], Xy[1], Xy[2], model)
        
results_df = pd.DataFrame(results)

In [ ]:
# training scores
results_df[results_df['test_train'] == 'train'].sort_values('score', ascending=False)

In [ ]:
# training scores
results_df[results_df['test_train'] == 'test'].sort_values('score', ascending=False)

In [ ]:
results_df.write_csv('data/benchmark_model_scores.csv')